In [ ]:
!pip install datasets
!pip install requests beautifulsoup4 pandas
!pip install beautifulsoup4 requests pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the base URL of the website
base_url = 'https://dailyhindustanexpress.com/'

# Initialize lists to store data
titles = []
urls = []
contents = []

# Define a function to fetch articles from a page
def fetch_articles_from_page(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all articles on the current page
    for article in soup.find_all('article'):
        # Extract the title and URL
        title_tag = article.find('h3', {'class': 'jeg_post_title'})  # Adjust the tag as necessary
        if title_tag:
            title = title_tag.get_text(strip=True)
            url = title_tag.find('a')['href']

            # Fetch the article content
            article_response = requests.get(url)
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            # Extract the content
            content_tag = article_soup.find('div', {'class': 'content-inner'})  # Adjust the class as necessary
            content = content_tag.get_text(strip=True) if content_tag else 'No content found'

            # Append the data to lists
            titles.append(title)
            urls.append(url)
            contents.append(content)

# Fetch the first page
current_page_url = base_url

# Loop through all pages using the "next" button
while current_page_url:
    # Fetch articles from the current page
    fetch_articles_from_page(current_page_url)

    # Fetch the next page URL
    response = requests.get(current_page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    next_page_tag = soup.find('a', {'class': 'page_nav next'})  # Adjust the class based on the inspect code
    if next_page_tag:
        next_page_url = next_page_tag['href']
        current_page_url = next_page_url  # Update the URL for the next page
    else:
        current_page_url = None  # Stop the loop if there is no next page

# Create a DataFrame from the lists
df = pd.DataFrame({
    'Title': titles,
    'URL': urls,
    'Content': contents
})

# Save DataFrame to CSV
df.to_csv('HindustanExpress.csv', index=False)
print("Scraping completed")